In [2]:
import pandas as pd
from fastparquet import write
df=pd.read_parquet('comment_id.parquet',engine='fastparquet')

df.insert(2,"count",1)
print(df)

df = df.groupby(['아이디','제목']).size().reset_index(name='count')
print(df.loc[df['count']==2])
print(df)

         제목       아이디  count
0      인생존망  fkqp****      1
1      인생존망  kjh2****      1
2      인생존망  mmoa****      1
3      인생존망  ycg0****      1
4      인생존망  dbgp****      1
...     ...       ...    ...
48818  순정만화  zezeㅋㄷㅋㄷ      1
48819  순정만화       8차원      1
48820  순정만화       신수진      1
48821  순정만화       서헤빗      1
48822  순정만화      가을하늘      1

[3046741 rows x 3 columns]
         아이디          제목  count
23         -   11년 후 우리는      2
26         -        CCTV      2
27         -      Dr.브레인      2
28         -        GONE      2
32         -  가내수공업 Ent.      2
...      ...         ...    ...
1699880   히스        파륜아이      2
1699890  히에이        코알랄라      2
1699896   히피         제비전      2
1699931  힘들어        까야제맛      2
1699935  힙합걸         에비츄      2

[264432 rows x 3 columns]
                   아이디              제목  count
0                  불렉션            실버볼1      1
1                    찐            이장본색      1
2             Lr매ㄱr고ㅍr        반짝반짝 컬링부      1
3                chahy       

In [3]:
df = df.pivot('아이디', '제목', 'count').fillna(0)
print(df)

제목             0.0MHz  0.1초의 설렘  100%의 그녀  ...  히어로즈 오브 레전드  힘내요 일본!...  힙한남자
아이디                                        ...                               
       불렉션        0.0       0.0       0.0  ...          0.0         0.0   0.0
       찐          0.0       0.0       0.0  ...          0.0         0.0   0.0
 Lr매ㄱr고ㅍr         0.0       0.0       0.0  ...          0.0         0.0   0.0
 chahy            0.0       0.0       0.0  ...          0.0         0.0   0.0
 곰지123            0.0       0.0       0.0  ...          0.0         0.0   0.0
...               ...       ...       ...  ...          ...         ...   ...
＊ㄴδㅁξ - - ª..     0.0       0.0       0.0  ...          0.0         0.0   0.0
＊오가＊              0.0       0.0       0.0  ...          0.0         0.0   0.0
＊좋은생각＊            0.0       0.0       0.0  ...          0.0         0.0   0.0
￡0ⓥe ⓘs...        0.0       0.0       0.0  ...          0.0         0.0   0.0
￡ΟνΕ　 Ιε 。。♡      0.0       0.0       0.0  ...          0.0     

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import operator

def similar_users(user_id, matrix, k=3):
    # create a df of just the current user
    user = matrix[matrix.index == user_id]
    
    # and a df of all other users
    other_users = matrix[matrix.index != user_id]
    
    # calc cosine similarity between user and each other user
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    # create list of indices of these users
    indices = other_users.index.tolist()
    
    # create key/values pairs of user index and their similarity
    index_similarity = dict(zip(indices, similarities))
    
    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    # grab k users off the top
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
    return users
    
# 사용자가 최근 본 웹툰 3개 입력
user_webtoon_input=['인생존망','독립일기','유미의 세포들']
# 전체 데이터 프레임에 최근 본 웹툰 3개의 가중치값을 1로 두어 행을 추가
df.loc['user_input']={'인생존망':1,'독립일기':1,'유미의 세포들':1}
df=df.fillna(0)

current_user = "user_input"
# try it out
similar_user_indices = similar_users(current_user, df)
print(similar_user_indices)
#=> [30773, 39021, 45603]

['72ro****', 'wh07****', 'dam3****']


In [7]:
print(df)

제목            0.0MHz  0.1초의 설렘  100%의 그녀  ...  히어로즈 오브 레전드  힘내요 일본!...  힙한남자
아이디                                       ...                               
       불렉션       0.0       0.0       0.0  ...          0.0         0.0   0.0
       찐         0.0       0.0       0.0  ...          0.0         0.0   0.0
 Lr매ㄱr고ㅍr        0.0       0.0       0.0  ...          0.0         0.0   0.0
 chahy           0.0       0.0       0.0  ...          0.0         0.0   0.0
 곰지123           0.0       0.0       0.0  ...          0.0         0.0   0.0
...              ...       ...       ...  ...          ...         ...   ...
＊오가＊             0.0       0.0       0.0  ...          0.0         0.0   0.0
＊좋은생각＊           0.0       0.0       0.0  ...          0.0         0.0   0.0
￡0ⓥe ⓘs...       0.0       0.0       0.0  ...          0.0         0.0   0.0
￡ΟνΕ　 Ιε 。。♡     0.0       0.0       0.0  ...          0.0         0.0   0.0
user_input       0.0       0.0       0.0  ...          0.0         0.0   0.0

In [27]:
def recommend_item(user_index, similar_user_indices, matrix):
    
    # load vectors for similar users
    similar_users = matrix[matrix.index.isin(similar_user_indices)]
    # calc avg ratings across the 3 similar users
    similar_users = similar_users.mean(axis=0)
    # convert to dataframe so its easy to sort and filter
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
    
    # load vector for the current user
    user_df = matrix[matrix.index == user_index]
    # transpose it so its easier to filter
    user_df_transposed = user_df.transpose()
    # rename the column as 'rating'
    user_df_transposed.columns = ['rating']
    # remove any rows without a 0 value. webtoon not watched yet
    user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
    # generate a list of webtoons the user has not seen
    webtoon_unseen = user_df_transposed.index.tolist()
    
    # filter avg ratings of similar users for only webtoon the current user has not seen
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(webtoon_unseen)]
    # order the dataframe
    similar_users_df_ordered = similar_users_df_filtered.sort_values(by=['mean'], ascending=False)

    return similar_users_df_ordered #items
# try it out
recommend_item(current_user, similar_user_indices, df)

,mean
제목,
놓정동화,0.666667
슈퍼 시크릿,0.333333
삼국지톡,0.333333
아이돌 연구소,0.333333
덴마,0.333333
...,...
마왕비 납시오,0.000000
마야비,0.000000
마애(磨崖),0.000000
